In [ ]:
import sys
sys.path.append('..')

In [ ]:
from modular import data_setup, engine, models, utils, helper
import torch
from pathlib import Path

In [ ]:
# Model setup
model, transforms = models.create_vit_model(5, 42)

In [ ]:
# Data setup

train_dir = '../data/train'
test_dir = '../data/test'

train_dataloader, test_dataloader, class_names, targets = data_setup.create_dataloaders(train_dir, test_dir, transforms)

In [ ]:
# Setup loss function and optimizer
LR = 0.001
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
EPOCHS = 3
results = engine.train(model, train_dataloader, test_dataloader, optimizer=optimizer, loss_fn=loss_fn, epochs=EPOCHS, device='cpu')

In [ ]:
helper.plot_results(results, EPOCHS)

In [ ]:
fig, ax = helper.predict_and_plot_confusion_matrix(test_dataloader, model, class_names, targets)

In [ ]:
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))
df = helper.predict_and_store(test_data_paths, model, transforms, class_names)
df

In [ ]:
print("Averagetime for prediction", round(df['time_for_pred'].mean(),4))

In [ ]:
# Predict and plot with a random sample

val_path = Path('../data/val')
engine.predict_with_samples(val_path, model, transforms, class_names)